In [2]:
import pandas as pd

In [3]:
share_energy_df = pd.read_csv('CSVS\\01 renewable-share-energy.csv')
consumption_df = pd.read_csv('CSVS\\02 modern-renewable-energy-consumption.csv')
production_df = pd.read_csv('CSVS\\03 modern-renewable-prod.csv')
solar_cap_df = pd.read_csv('CSVS\\13 installed-solar-PV-capacity.csv')

In [ ]:
merged_df = production_df.merge(consumption_df , on = ['Entity', 'Year', 'Code'], how='outer')
merged_df = merged_df.merge(share_energy_df , on = ['Entity', 'Year', 'Code'], how='outer')
merged_df = merged_df.merge(solar_cap_df, on = ['Entity', 'Year', 'Code'], how='outer')

,Entity,Code,Year,Electricity from wind (TWh),Electricity from hydro (TWh),Electricity from solar (TWh),Other renewables including bioenergy (TWh),Geo Biomass Other - TWh,Solar Generation - TWh,Wind Generation - TWh,Hydro Generation - TWh,Renewables (% equivalent primary energy),Solar Capacity
0,Afghanistan,AFG,2000,0.0,0.31,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,2001,0.0,0.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,2002,0.0,0.56,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,2003,0.0,0.63,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,2004,0.0,0.56,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Normalización de los datos y creación de las tablas

In [6]:
# Crear la tabla de países/regiones sin duplicados
countries_df = merged_df[['Entity', 'Code']].drop_duplicates().reset_index(drop=True)
countries_df['country_id'] = countries_df.index + 1
countries_df = countries_df[['country_id', 'Entity', 'Code']]

In [7]:
# Crear el DataFrame de continentes utilizando diccionario por registro
data_continentes = [
    {'Continent': 'Africa', 'continent_id': 1},
    {'Continent': 'North America', 'continent_id': 2},
    {'Continent': 'South America', 'continent_id': 3},
    {'Continent': 'Oceania', 'continent_id': 4},
    {'Continent': 'Europe', 'continent_id': 5},
    {'Continent': 'Asia', 'continent_id': 6},
    {'Continent': 'Other', 'continent_id': 7}
]

# Crear el DataFrame
continentes_df = pd.DataFrame(data_continentes)
continentes_df = continentes_df[['continent_id', 'Continent']]

# Usamos un csv con los países y su respectivo continente
continentes_paises_df = pd.read_csv('CSVS\\18 Countries by continents.csv')
continentes_paises_df = continentes_paises_df.rename(columns={'Country':'Entity'})

# Asignar el continente al país usando el csv
countries_df = countries_df.merge(continentes_paises_df, on='Entity', how='left')

# Añadimos los continentes manualmente en algunos de los países restantes
countries_df.loc[countries_df['Entity'].isin(['Bermuda', 'Greenland', 'Puerto Rico']), 'Continent'] = 'North America'
countries_df.loc[countries_df['Entity'].isin(['Aruba']), 'Continent'] = 'South America'
countries_df.loc[countries_df['Entity'].isin(['Samoa', 'Guam']), 'Continent'] = 'Oceania'
countries_df.loc[countries_df['Entity'].isin(['Burkina Faso', "Cote d'Ivoire", 'Eswatini']), 'Continent'] = 'Africa'
countries_df.loc[countries_df['Entity'].isin(['Kosovo', 'North Macedonia']), 'Continent'] = 'Europe' 
countries_df.loc[countries_df['Entity'].isin(['Myanmar', 'Macao', 'Palestine']), 'Continent'] = 'Asia'

In [8]:
# Crear una nueva columna 'id_continente' y asignar el continente
countries_df = countries_df.merge(continentes_df, on = 'Continent', how = 'left')
countries_df = countries_df.drop(columns='Continent')
countries_df = countries_df.reset_index(drop=True)
countries_df['continent_id'] = countries_df['continent_id'].fillna(7).astype(int)


In [9]:
# Añadir las llaves fóraneas de país y contiente al df original

# Unir con el dataframe original para asignar country_id llave foránea
values_df = merged_df.merge(countries_df, on=['Entity', 'Code'], how='left')
values_df = values_df.drop(columns=['Entity', 'Code'])


In [10]:
# Crear la tabla de años sin duplicados
years_df = merged_df[['Year']].drop_duplicates().reset_index(drop = True)
years_df.insert(0, 'year_id', range(1, len(years_df) + 1))

#Unir con el dataframe original para asignar year_id como llave foránea
values_df = values_df.merge(years_df, on='Year', how='left')
values_df = values_df.drop(columns=['Year'])

In [11]:
# Datos comunes y preparación del values_df
foreign_keys = ['country_id', 'year_id']

# Cambiamos los valor nulos por 0
values_df = values_df.fillna(0)

In [12]:
# Crear la tabla de consumo conservando las llaves foráneas
consumption_df = values_df[['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh'] + foreign_keys]
consumption_df.index += 1

In [13]:
# Crear la tabla de producción conservando las llaves foráneas
production_df = values_df[['Electricity from wind (TWh)', 'Electricity from hydro (TWh)', 'Electricity from solar (TWh)', 'Other renewables including bioenergy (TWh)'] + foreign_keys]
production_df.index += 1

In [14]:
# Crear la tabla de porcentaje de energía renovable respecto al total conservando las llaves foráneas
renewable_percent_df = values_df[['Renewables (% equivalent primary energy)'] + foreign_keys]
renewable_percent_df.index += 1

In [15]:
# Crear la tabla de capacidad de energía solar conservando las llaves foráneas
solar_capacity_df = values_df[['Solar Capacity'] + foreign_keys]
solar_capacity_df.index += 1

In [16]:
# Exportar los CSVs normalizados
countries_df.to_csv('entities.csv', index=False)
years_df.to_csv('years.csv', index=False)
continentes_df.to_csv('regions.csv',index=False)
consumption_df.to_csv('consumption.csv')
production_df.to_csv('production.csv')
renewable_percent_df.to_csv('renewable_percent.csv')
solar_capacity_df.to_csv('solar_capacity.csv')

In [ ]:
"""
-- 1. Tabla para regions.csv
CREATE TABLE regions (
    region_id INT PRIMARY KEY AUTO_INCREMENT,
    region VARCHAR(255) NOT NULL
);

-- 2. Tabla para entities.csv
CREATE TABLE entities (
    entity_id INT PRIMARY KEY AUTO_INCREMENT,
    entity VARCHAR(255) NOT NULL,
    code VARCHAR(10),
    region_id INT,
    FOREIGN KEY (region_id) REFERENCES regions(region_id) ON DELETE SET NULL ON UPDATE CASCADE
);

-- 3. Tabla para years.csv
CREATE TABLE years (
    year_id INT PRIMARY KEY AUTO_INCREMENT,
    year INT NOT NULL UNIQUE
);

-- 4. Tabla para renewable_energy_consumption.csv
CREATE TABLE renewable_energy_consumption (
    id INT PRIMARY KEY AUTO_INCREMENT,
    bioenergy_TWh FLOAT NOT NULL,
    solar_TWh FLOAT NOT NULL,
    wind_TWh FLOAT NOT NULL,
    hydro_TWh FLOAT NOT NULL,
    entity_id INT NOT NULL,
    year_id INT NOT NULL,
    FOREIGN KEY (entity_id) REFERENCES entities(entity_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (year_id) REFERENCES years(year_id) ON DELETE CASCADE ON UPDATE CASCADE
);

-- 5. Tabla para renewable_energy_production.csv
CREATE TABLE renewable_energy_production (
    id INT PRIMARY KEY AUTO_INCREMENT,
    wind_TWh FLOAT NOT NULL,
    hydro_TWh FLOAT NOT NULL,
    solar_TWh FLOAT NOT NULL,
    bioenergy_TWh FLOAT NOT NULL,
    entity_id INT NOT NULL,
    year_id INT NOT NULL,
    FOREIGN KEY (entity_id) REFERENCES entities(entity_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (year_id) REFERENCES years(year_id) ON DELETE CASCADE ON UPDATE CASCADE
);

-- 6. Tabla para renewable_percent.csv
CREATE TABLE renewable_percent (
    id INT PRIMARY KEY AUTO_INCREMENT,
    renewables_percent FLOAT NOT NULL,
    entity_id INT NOT NULL,
    year_id INT NOT NULL,
    FOREIGN KEY (entity_id) REFERENCES entities(entity_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (year_id) REFERENCES years(year_id) ON DELETE CASCADE ON UPDATE CASCADE
);

-- 7. Tabla para solar_capacity.csv
CREATE TABLE solar_capacity (
    id INT PRIMARY KEY AUTO_INCREMENT,
    solar_capacity FLOAT NOT NULL,
    entity_id INT NOT NULL,
    year_id INT NOT NULL,
    FOREIGN KEY (entity_id) REFERENCES entities(entity_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (year_id) REFERENCES years(year_id) ON DELETE CASCADE ON UPDATE CASCADE
);

-- Ejemplos Select's
SELECT * FROM entities;
SELECT * FROM years;
SELECT * FROM regions;
SELECT * FROM consumption;
SELECT * FROM production;
SELECT * FROM renewable_percent;
SELECT * FROM solar_capacity;
"""

"\n-- 1. Tabla para entities.csv\nCREATE TABLE entities (\n    entity_id INT PRIMARY KEY,\n    entity VARCHAR(255) NOT NULL,\n    code VARCHAR(10),\n    region_id INT,\n    FOREIGN KEY (region_id) REFERENCES regions(region_id)\n);\n\n-- 2. Tabla para years.csv\nCREATE TABLE years (\n    year_id INT PRIMARY KEY,\n    year INT NOT NULL\n);\n\n-- 3. Tabla para regions.csv\nCREATE TABLE regions (\n    region_id INT PRIMARY KEY,\n    region VARCHAR(255) NOT NULL\n);\n\n-- 4. Tabla para consumption.csv\nCREATE TABLE renewable_energy_consumption (\n    id INT PRIMARY KEY AUTO_INCREMENT,\n    bioenergy_TWh FLOAT NOT NULL,\n    solar_TWh FLOAT NOT NULL,\n    wind_TWh FLOAT NOT NULL,\n    hydro_TWh FLOAT NOT NULL,\n    entity_id INT,\n    region_id INT,\n    year_id INT,\n    FOREIGN KEY (entity_id) REFERENCES entities(entity_id),\n    FOREIGN KEY (region_id) REFERENCES regions(region_id),\n    FOREIGN KEY (year_id) REFERENCES years(year_id)\n);\n\n-- 5. Tabla para production.csv\nCREATE TABLE r